## Algorithm Complexity: Building Fast Queries on a CSV

In this project, we assume the position in which we own an online laptop store and want to build a way to answer a few different business questions about our inventory. For this purpose, we will use data recopilated in a CSV dataset about [laptop prices from Kaggle](https://www.kaggle.com/ionaskel/laptop-prices).

We will use this exercise to analyse the __time and space complexity__ of the different algorithms implemented, and the ways in which it is possible to improve these algorithms to make them more time effectivily.

### Step 1: Exploring the dataset

As first step, we provide a short description of the different rows inside the dataset:

    - ID: A unique identifier for the laptop.
    - Company: The name of the company that produces the laptop.
    - Product: The name of the laptop.
    - TypeName: The type of laptop.
    - Inches: The size of the screen in inches.
    - ScreenResolution: The resolution of the screen.
    - CPU: The laptop CPU.
    - RAM: The amount of RAM in the laptop.
    - Memory: The size of the hard drive.
    - GPU: The graphics card name.
    - OpSys: The name of the operating system.
    - Weight: The laptop weight.
    - Price: The price of the laptop.


In [1]:
import csv

with open('laptops.csv', encoding='UTF-8') as file:
    rows = list(csv.reader(file))
    header = rows[0]
    rows = rows[1:]
    
print('Header row:\n', header)
for i in range(5):
    print('\nNext row:\n', rows[i])

Header row:
 ['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']

Next row:
 ['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339']

Next row:
 ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898']

Next row:
 ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575']

Next row:
 ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537']

Next row:
 ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Pan

The goal of this project is to create a class that represents our inventory. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

Some of the queries we will want to answer are:

    - Given a laptop id, find the corresponding data.
    - Given an amount of money, find whether there are two laptops whose total price is that given amount.
    - Identify all laptops whose price falls within a given budget.

### Step 2: Implementation of the basis of the new class

In [2]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='UTF-8') as file:
            rows = list(csv.reader(file))
            self.header = rows[0]
            self.rows = rows[1:]
            for row in self.rows:
                row[-1] = int(row[-1])

In [3]:
# Testing the new class
inventory = Inventory('laptops.csv')
print(inventory.header)
print(len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


### Step 3: Implementation of the queries

The queries will be implemented as improvements to the `Inventory` class. For this reason, in the next cells the class will be improved with the addition of new methods and features.

#### 3.1. Finding a laptop from the ID

The first thing that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer comes to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

In [4]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='UTF-8') as file:
            rows = list(csv.reader(file))
            self.header = rows[0]
            self.rows = rows[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
        return None

In [5]:
# Testing the class
inventory = Inventory('laptops.csv')
assert inventory.get_laptop_from_id('3362737') is not None, "Wrong test 1"
assert inventory.get_laptop_from_id('3362736') is None, "Wrong test 2"
print(inventory.get_laptop_from_id('3362737'))
print(inventory.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


The algorithm inside the method `get_laptop_from_id()` requires us to look at every single row to find the one that we are looking for (or decide that such a row does not exist). This algorithm has __time complexity O(R)__ where R is the number of rows.

We can solve this problem more efficiently by preprocessing the data. Using a set, we can check in constant time whether a given identifier exists. However, we don't just want to know if it exists, we also want to retrieve the remaining row information. Therefore, we will use a dictionary instead of a set. Dictionaries have the same fast lookup properties that sets have, but allow us to associate values to the keys.

In [6]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='UTF-8') as file:
            rows = list(csv.reader(file))
            self.header = rows[0]
            self.rows = rows[1:]
            self.id_to_row = dict()
            for row in self.rows:
                row[-1] = int(row[-1])
                self.id_to_row[row[0]] = row
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None

In [7]:
# Testing the class
inventory = Inventory('laptops.csv')
assert inventory.get_laptop_from_id_fast('3362737') is not None, "Wrong test 1"
assert inventory.get_laptop_from_id_fast('3362736') is None, "Wrong test 2"
print(inventory.get_laptop_from_id_fast('3362737'))
print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


This new implementation in the method `get_laptop_from_id_fast()` has __time complexity O(1)__. It does so by using more memory to store the `self.id_to_row` dictionary and using a bit more time creating an instance (because it needs to create the dictionary).

Let's experiment to compare the performance of the two methods. The idea is to generate random IDs using the random module. Then, use both methods to lookup these same IDs. We will use the time module to measure the execution time of each lookup and, for each method, add all times together.

In [8]:
import random
import time

# Creating 10,000 random ids
random.seed(42)
ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

# Creating the instance
inventory = Inventory('laptops.csv')

# Performance for slow method with time complexity O(R)
total_time_no_dict = 0
for identifier in ids:
    start = time.time()
    inventory.get_laptop_from_id(identifier)
    end = time.time()
    total_time_no_dict += end - start
    
# Performance for fast method with time complexity O(1)
total_time_dict = 0
for identifier in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(identifier)
    end = time.time()
    total_time_dict += end - start
    
print('Total querying time slow performance: {0:.4f}s'.format(total_time_no_dict))
print('Total querying time fast performance: {0:.4f}s'.format(total_time_dict))

Total querying time slow performance: 0.9104s
Total querying time fast performance: 0.0049s


We can see a significant improve in performance. The new method is about 185 times faster for this input size.


#### 3.2. Two laptop promotion

Sometimes, the store offers a promotion where we give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. This means that, even if there is leftover money, it cannot be used anymore.

Whenever we issue a gift card, we want to make sure that there is at least one way to spend it in full to avoid unhappy customers. In other words, before issuing a gift card for D dollars, we want to make sure that either there is a laptop that costs exactly D dollars or two laptops whose costs add up to precisely D dollars.

For that, we will write a function that, given a dollar amount, checks wheher it is possible to spend precisely that amount by purchasing up to two laptops.

In [9]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='UTF-8') as file:
            rows = list(csv.reader(file))
            self.header = rows[0]
            self.rows = rows[1:]
            self.id_to_row = dict()
            for row in self.rows:
                row[-1] = int(row[-1])
                self.id_to_row[row[0]] = row
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if dollars == row[-1]:
                return True
            for row2 in self.rows:
                if dollars == row[-1] + row2[-1]:
                    return True
        return False

In [10]:
# Testing the class
inventory = Inventory('laptops.csv')
assert inventory.check_promotion_dollars(1000) is True, "Wrong test 1"
assert inventory.check_promotion_dollars(442) is False, "Wrong test 2"
print(inventory.check_promotion_dollars(1000))
print(inventory.check_promotion_dollars(442))

True
False


The algorithm in the method `check_promotion_dollars()` has __time complexity O(R^2)__ where R is the number of rows. As before, we can preprocess data to make our code run faster.

Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. Then we can check in constant time whether there is a laptop with a given price.

To check if there is a pair of laptops whose sum adds up to the given value, we can take into consideration that `sum = laptop_price_1 + laptop_price_2`, i.e., `laptop_price_2 = sum - laptop_price_1`. With that, we can check in constant time wheter there is a laptop with a price equals to the difference between the given value and the other laptop's price.  

In [11]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='UTF-8') as file:
            rows = list(csv.reader(file))
            self.header = rows[0]
            self.rows = rows[1:]
            self.id_to_row = dict()
            self.prices = set()
            for row in self.rows:
                row[-1] = int(row[-1])
                self.id_to_row[row[0]] = row
                self.prices.add(row[-1])
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if dollars == row[-1]:
                return True
            for row2 in self.rows:
                if dollars == row[-1] + row2[-1]:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False

In [12]:
# Testing the class
inventory = Inventory('laptops.csv')
assert inventory.check_promotion_dollars_fast(1000) is True, "Wrong test 1"
assert inventory.check_promotion_dollars_fast(442) is False, "Wrong test 2"
print(inventory.check_promotion_dollars_fast(1000))
print(inventory.check_promotion_dollars_fast(442))

True
False


The algorithm for the new method `check_promotion_dollars_fast()` has a __time complexity O(R)__ where R is the number of rows. It does so by using more memory to store the `self.prices` set and using a bit more time creating an instance (because it needs to create the set).

Let's now compare the performance of the last two functions that we wrote.

In [13]:
# Creating 100 random values to check
random.seed(42)
prices = [random.randint(100, 5000) for _ in range(100)]

# Creating the instance
inventory = Inventory('laptops.csv')

# Performance for slow method with time complexity O(R^2)
total_time_no_set = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += end - start
    
# Performance for fast method with time complexity O(R)
total_time_set = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += end - start
    
print('Total querying time slow performance: {0:.4f}s'.format(total_time_no_set))
print('Total querying time fast performance: {0:.4f}s'.format(total_time_set))

Total querying time slow performance: 1.8304s
Total querying time fast performance: 0.0007s


We can see a significant improve in performance. The new method is about 2336 times faster for this input size.


#### 3.2. Finding laptops within a budget

We want to write a method that efficiently answers the query: Given a budget of D dollars, find all laptops whose price it at most D.

If we sort all laptops by price, we can use binary search to identify the first laptop in the sorted list with a price larger than D. We need to make sure that our binary search finds the first one on the list. Then, the result of the query will consist of all laptops whose index in the sorted list is smaller than the index of the first laptop whose price is higher than D dollars.

In [14]:
class Inventory:
    
    def __init__(self, csv_filename):
        with open(csv_filename, encoding='UTF-8') as file:
            rows = list(csv.reader(file))
            self.header = rows[0]
            self.rows = rows[1:]
            self.id_to_row = dict()
            self.prices = set()
            for row in self.rows:
                row[-1] = int(row[-1])
                self.id_to_row[row[0]] = row
                self.prices.add(row[-1])
            self.rows_by_price = sorted(self.rows, key=lambda row: row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if laptop_id == row[0]:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if dollars == row[-1]:
                return True
            for row2 in self.rows:
                if dollars == row[-1] + row2[-1]:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                       
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]                       
            if price < target_price:                           
                range_start = range_middle + 1             
            else:                                          
                range_end = range_middle - 1 
        price = self.rows_by_price[range_start][-1]
        if price != target_price:                  
            return -1                                      
        return range_start+1

In [15]:
# Testing the class
inventory = Inventory('laptops.csv')
# assert inventory.find_first_laptop_more_expensive(1000) is True, "Wrong test 1"
# assert inventory.find_first_laptop_more_expensive(10000) is False, "Wrong test 2"
print(inventory.find_first_laptop_more_expensive(1000))
print(inventory.find_first_laptop_more_expensive(10000))

683
-1


As a binary search algorithm modification, our method `find_first_laptop_more_expensive` has a __O(log(R)) time complexity__ where R is the number of rows.

If we want to push this project further, we can think about the following queries:

   - Imagine that we extend our budget query to take as input a range of prices, min_price and max_price, rather than a single price. Write a query that finds all laptops whose price is in the given range.
   - Sometimes, a customer wants a laptop with some characteristics such as, for instance, 8GB or RAM and a 256GB hard drive. It would be interesting for those customers to provide a way to find the cheapest laptop that matches the desired characteristics.
